In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df= pd.read_csv('../data/income_data.csv')

### Transformacion de variables

In [30]:
# Aplicar logaritmo a capital-gain y capital-loss
df_procc['capital_gain'] = np.log1p(df_procc['capital-gain'])
df_procc['capital_loss'] = np.log1p(df_procc['capital-loss'])

# Eliminar las columnas originales (sin asignar el resultado)
df_procc.drop(columns=['capital-gain', 'capital-loss'], inplace=True)

### Imputacion de valores faltantes

In [29]:
df_procc = df.copy()

df_procc['workclass'] = df_procc['workclass'].replace('?', np.nan)
df_procc['workclass'].fillna(df_procc['workclass'].mode()[0], inplace=True)

df_procc['occupation'] = df_procc['occupation'].replace('?', np.nan)
df_procc['occupation'].fillna(df_procc['occupation'].mode()[0], inplace=True)

print(df_procc["workclass"].value_counts())
print(df_procc["occupation"].value_counts())


workclass
Private             36705
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: count, dtype: int64
occupation
Prof-specialty       8981
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: count, dtype: int64


C:\Users\Victor Casas\AppData\Local\Temp\ipykernel_16268\3813693468.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_procc['workclass'].fillna(df_procc['workclass'].mode()[0], inplace=True)
C:\Users\Victor Casas\AppData\Local\Temp\ipykernel_16268\3813693468.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

In [32]:
df_procc

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,hours-per-week,native-country,income,capital_gain,capital_loss
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,40,United-States,<=50K,0.000000,0.0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,50,United-States,<=50K,0.000000,0.0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,40,United-States,>50K,0.000000,0.0
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,40,United-States,>50K,8.947546,0.0
4,18,Private,103497,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,30,United-States,<=50K,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,38,United-States,<=50K,0.000000,0.0
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,0.000000,0.0
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,40,United-States,<=50K,0.000000,0.0
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,20,United-States,<=50K,0.000000,0.0


#### Eliminacion de outliers

In [36]:
def remove_outliers_iqr(df_procc, columns):
    """Quitando los valores atípicos con el método del rango intercuartílico, 
    se podría ser menos estricto con los outliers en vez de multiplicar el IQR por 1.5 por un número mayor
    
    Parámetros:
    - df_procc: Dataframe con los valores nulos y columnas menos relevantes ya eliminados
    - columns: nombres de las columnas del df previamente seleccionadas a las cuales se les eliminarán los outliers

    Retorna: devuelve el dataframe con los outliers eliminados de las columnas seleccionadas
   """
    for col in columns:
        Q1 = df_procc[col].quantile(0.25)
        Q3 = df_procc[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_procc = df_procc[(df_procc[col] >= lower_bound) & (df_procc[col] <= upper_bound)]
    return df_procc


elim_outliers = ['fnlwgt']  


df_procc = remove_outliers_iqr(df_procc, elim_outliers)


In [37]:
df_procc.shape

(47389, 15)

### Codificar variables

In [38]:
#importamos one-hot encoding para las variables categóricas
from sklearn.preprocessing import OneHotEncoder


df_procc["gender"]=df_procc["gender"].replace({"Male": 0, "Female": 1})
df_procc["income"]=df_procc["income"].replace({"<=50K": 0, ">50K": 1})

df_procc

C:\Users\Victor Casas\AppData\Local\Temp\ipykernel_16268\1467086059.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_procc["gender"]=df_procc["gender"].replace({"Male": 0, "Female": 1})
C:\Users\Victor Casas\AppData\Local\Temp\ipykernel_16268\1467086059.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_procc["gender"]=df_procc["gender"].replace({"Male": 0, "Female": 1})
C:\Users\Victor Casas\AppData\Local\Temp\ipykernel_16268\1467086059.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,hours-per-week,native-country,income,capital_gain,capital_loss
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,0,40,United-States,0,0.000000,0.0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,0,50,United-States,0,0.000000,0.0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,0,40,United-States,1,0.000000,0.0
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,0,40,United-States,1,8.947546,0.0
4,18,Private,103497,Some-college,10,Never-married,Prof-specialty,Own-child,White,1,30,United-States,0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,1,38,United-States,0,0.000000,0.0
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,0,40,United-States,1,0.000000,0.0
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,1,40,United-States,0,0.000000,0.0
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,0,20,United-States,0,0.000000,0.0


In [39]:
from sklearn.preprocessing import OneHotEncoder

# Inicializar el codificador
encoder = OneHotEncoder(sparse_output=False, drop='first')

# Columnas categóricas a codificar
categorical_cols = ['workclass', 'marital-status', 'occupation', 'relationship', 'race']

# Ajustar y transformar
encoded_array = encoder.fit_transform(df_procc[categorical_cols])

# Crear DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded_array,
                          columns=encoder.get_feature_names_out(categorical_cols),
                          index=df_procc.index)

# Concatenar al DataFrame original eliminando las originales
df_encoded = pd.concat([df_procc.drop(categorical_cols, axis=1), encoded_df], axis=1)


In [40]:
education_order = {
    'Preschool': 1,
    '1st-4th': 2,
    '5th-6th': 3,
    '7th-8th': 4,
    '9th': 5,
    '10th': 6,
    '11th': 7,
    '12th': 8,
    'HS-grad': 9,
    'Some-college': 10,
    'Assoc-voc': 11,
    'Assoc-acdm': 12,
    'Bachelors': 13,
    'Masters': 14,
    'Prof-school': 15,
    'Doctorate': 16
}
df_encoded['education_encoded'] = df_encoded['education'].map(education_order)
df_encoded= df_encoded.drop(columns=['education'], axis=1)

In [41]:
# Eliminamos native-country
df_encoded = df_encoded.drop(columns=['native-country'], axis=1)

In [42]:
df_encoded

,age,fnlwgt,educational-num,gender,hours-per-week,income,capital_gain,capital_loss,workclass_Local-gov,workclass_Never-worked,...,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_Other,race_White,education_encoded
0,25,226802,7,0,40,0,0.000000,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,7
1,38,89814,9,0,50,0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9
2,28,336951,12,0,40,1,0.000000,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,12
3,44,160323,10,0,40,1,8.947546,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10
4,18,103497,10,1,30,0,0.000000,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,257302,12,1,38,0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,12
48838,40,154374,9,0,40,1,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9
48839,58,151910,9,1,40,0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,9
48840,22,201490,9,0,20,0,0.000000,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,9


In [43]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47389 entries, 0 to 48841
Data columns (total 44 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   age                                   47389 non-null  int64  
 1   fnlwgt                                47389 non-null  int64  
 2   educational-num                       47389 non-null  int64  
 3   gender                                47389 non-null  int64  
 4   hours-per-week                        47389 non-null  int64  
 5   income                                47389 non-null  int64  
 6   capital_gain                          47389 non-null  float64
 7   capital_loss                          47389 non-null  float64
 8   workclass_Local-gov                   47389 non-null  float64
 9   workclass_Never-worked                47389 non-null  float64
 10  workclass_Private                     47389 non-null  float64
 11  workclass_Self-emp-i

In [46]:
df_encoded.isnull().sum()

age                                     0
fnlwgt                                  0
educational-num                         0
gender                                  0
hours-per-week                          0
income                                  0
capital_gain                            0
capital_loss                            0
workclass_Local-gov                     0
workclass_Never-worked                  0
workclass_Private                       0
workclass_Self-emp-inc                  0
workclass_Self-emp-not-inc              0
workclass_State-gov                     0
workclass_Without-pay                   0
marital-status_Married-AF-spouse        0
marital-status_Married-civ-spouse       0
marital-status_Married-spouse-absent    0
marital-status_Never-married            0
marital-status_Separated                0
marital-status_Widowed                  0
occupation_Armed-Forces                 0
occupation_Craft-repair                 0
occupation_Exec-managerial        

### Eliminacion de duplicados

In [49]:
df_encoded.duplicated().sum()
df_encoded.drop_duplicates(inplace=True)
df_encoded.duplicated().sum()


np.int64(0)

Guardamos el df final en un csv

In [52]:
import os

# Asegúrate de que la carpeta exista
output_dir = "C:/2cuatri_3/Proyecto prediccion/prueba1_EDA/data/data_out"
os.makedirs(output_dir, exist_ok=True)

# Guarda el archivo dentro de esa carpeta
df.to_csv(os.path.join(output_dir, "datos_limpios.csv"), index=False)
